In [1]:
import json
import pandas as pd
import geopandas as gp
import matplotlib as plt
import optparse
import sys
import os
from fiona.crs import from_epsg
import urllib.request
from shapely.ops import cascaded_union

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
## Download Street Centerline Data from NYC Open Data
url = 'https://data.cityofnewyork.us/api/geospatial/exjm-f27b?method=export&format=Shapefile'
urllib.request.urlretrieve(url,'Streets.zip')
! unzip Streets.zip -d data
files = ! ls data | grep '.shp' | grep 'geo_export_'
streets=gp.GeoDataFrame.from_file('data/'+files[0])
## Select only BX and BK Streets
BXSTREETS = streets[streets['borocode']=='2']
BKSTREETS = streets[streets['borocode']=='3']


Archive:  Streets.zip
  inflating: data/geo_export_d1a676ab-daf7-45e1-8906-ec85ad376e52.dbf  
  inflating: data/geo_export_d1a676ab-daf7-45e1-8906-ec85ad376e52.shp  
  inflating: data/geo_export_d1a676ab-daf7-45e1-8906-ec85ad376e52.shx  
  inflating: data/geo_export_d1a676ab-daf7-45e1-8906-ec85ad376e52.prj  


In [15]:
## Download NYC Transit Data
url = 'http://faculty.baruch.cuny.edu/geoportal/data/nyc_transit/nov2018/bus_routes_nyc_nov2018.zip'
urllib.request.urlretrieve(url,'Buses.zip')
! unzip Buses.zip -d data
busss='data/bus_routes_nyc_nov2018.shp'
bus=gp.GeoDataFrame.from_file(busss)
## Select only Bronx and Brooklyn Buses
BXBUS = bus[(bus.index >118) & (bus.index<205)]
BKBUS = bus[bus.index <=106]

Archive:  Buses.zip
 extracting: data/bus_routes_nyc_nov2018.cpg  
  inflating: data/bus_routes_nyc_nov2018.dbf  
  inflating: data/bus_routes_nyc_nov2018.pdf  
  inflating: data/bus_routes_nyc_nov2018.prj  
  inflating: data/bus_routes_nyc_nov2018.shp  
  inflating: data/bus_routes_nyc_nov2018.shx  
  inflating: data/bus_routes_nyc_nov2018_iso.xml  


In [18]:
## Download Borough Boundary Data (NYC OPEN DATA)
url='https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile'
urllib.request.urlretrieve(url,'Boundaries.zip')
! unzip Boundaries.zip -d data/Boundaries
files = ! ls data/Boundaries | grep '.shp' | grep 'geo_export_'
BOUNDARY = gp.GeoDataFrame.from_file("data/Boundaries/" + files[0])
BOUNDARYBK = BOUNDARY[BOUNDARY['boro_name'] == 'Brooklyn']
BOUNDARYBX = BOUNDARY[BOUNDARY['boro_name'] == 'Bronx']

Archive:  Boundaries.zip
  inflating: data/Boundaries/geo_export_00576701-d979-4b00-8cbc-bc2e3c2daaae.dbf  
  inflating: data/Boundaries/geo_export_00576701-d979-4b00-8cbc-bc2e3c2daaae.shp  
  inflating: data/Boundaries/geo_export_00576701-d979-4b00-8cbc-bc2e3c2daaae.shx  
  inflating: data/Boundaries/geo_export_00576701-d979-4b00-8cbc-bc2e3c2daaae.prj  


In [ ]:
url='https://data.cityofnewyork.us/api/geospatial/yfnk-k7r4?method=export&format=Shapefile'
urllib.request.urlretrieve(url,'CD.zip')
! unzip CD.zip -d data/CD
files = ! ls data/CD/ | grep '.shp' | grep 'geo_export_'
CD = gp.GeoDataFrame.from_file("data/CD/" + files[0])

In [ ]:
#url = "https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/bx_mappluto_18v1.zip"
#urllib.request.urlretrieve(url,'BronxPLUTO.zip')
#! unzip BronxPLUTO.zip -d data
#BXPLUTO = gp.GeoDataFrame.from_file("data/BXMapPLUTO.shp")


In [ ]:
## Download & Make GeoDataFrames of Zoning & Boundary shapefiles

#! rm 'Zoning.zip'
#url2 ='https://data.cityofnewyork.us/api/assets/FAC46E85-65FF-406F-BA16-E1039EF00B7E?download=true'
#urllib.request.urlretrieve(url2,'Zoning.zip')
#! unzip Zoning.zip -d data

#ZONING = gp.GeoDataFrame.from_file("data/nycgiszoningfeatures_201307_shp/nyzd.shp")
#SPZONING = gp.GeoDataFrame.from_file("data/nycgiszoningfeatures_201307_shp/nysp.shp")
#ZONING=ZONING.to_crs({'init': 'epsg:4326', 'no_defs': True})

In [22]:
## convert to same crs
BXBUS=BXBUS.to_crs({'init': 'epsg:4326', 'no_defs': True})
BKBUS=BKBUS.to_crs({'init': 'epsg:4326', 'no_defs': True})

In [23]:
BKboundary = gp.GeoSeries(cascaded_union(BKSTREETS.buffer(0.0001)))
BKboundarygdf= gp.GeoDataFrame(BKboundary)
BXboundary = gp.GeoSeries(cascaded_union(BXSTREETS.buffer(0.0001)))
BXboundarygdf= gp.GeoDataFrame(BXboundary)

In [24]:
BKboundarygdf.rename(columns={0:'geometry'},inplace=True)
BXboundarygdf.rename(columns={0:'geometry'},inplace=True)

In [25]:
buslength=pd.DataFrame(index=BKBUS.route_dir.values)
buslength['BK']=0.006
buslength['BX']=0.006
for k in BKBUS.route_dir.values:
    buslength['BK'][k] = gp.GeoSeries(cascaded_union\
                            ((BKBUS[BKBUS['route_dir']==k]).buffer(0.00015))).\
                            intersection(BKboundarygdf).area


In [ ]:
bxbuslength=pd.DataFrame(index=BXBUS.route_dir.values)
bxbuslength['BX']=0.006
for j in BXBUS.route_dir.values:
    bxbuslength['BX'][j] = gp.GeoSeries(cascaded_union\
                            ((BXBUS[BXBUS['route_dir']==j]).buffer(0.00015))).\
                            intersection(BXboundarygdf).area

In [ ]:
a = bxbuslength.sort_values('BX', ascending=False).head(20)

In [ ]:
bkbus['geometry'].buffer(0.00015).area.sum()

In [ ]:
buslength['Length'].sum()

In [ ]:
bkbus['geometry'].buffer(0.00015).area.sum()/boundarygdf.area.sum()

In [ ]:
buslength.sort_values(['Length'],ascending=False).head(10)

In [ ]:
type(buslength['Length']['B13_0'])

In [ ]:
gp.GeoSeries(cascaded_union((bkbus[bkbus['route_dir']=='B13_0']).buffer(0.00015))).intersection(boundarygdf).length

In [ ]:
b=busboundarygdf.intersection(boundarygdf)

In [ ]:
## The percentage of bus routes over 
b.area/boundarygdf.area

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
ax=boundarygdf.plot(ax=ax,color='pink')
b.plot(ax=ax)
plt.show()

In [ ]:
bkbus['length']=bkbus['geometry'].map(lambda x: [x.length])

In [ ]:
bushead = bkbus.sort_values(['length'],ascending=False).head(20)

In [ ]:
BXBUS['length']=BXBUS['geometry'].map(lambda x: [x.length])
bxhead = BXBUS.sort_values(['length'],ascending=False).head(20)

In [ ]:
bxhead

In [ ]:
bustail = bkbus.sort_values(['length'],ascending=False).tail(10)

In [ ]:
bkbus['buffer']=bkbus['geometry'].map(lambda x: [x.buffer(0.00015)])

In [ ]:
BOUNDARYBX = BOUNDARY[BOUNDARY['boro_name'] == 'Bronx']

In [ ]:
BX = BOUNDARYBX.intersection(ZONING)

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
ax=BXSTREETS.plot(ax=ax,color='pink')
bxhead.plot(ax=ax,column='route_id',cmap='rainbow',legend=True)
plt.title('Bronx Streets & Buses',size=20)
#ax.set_xlim(-73.94, -73.92)
#ax.set_ylim(40.65, 40.665)
plt.savefig('BronxStreets.png') 
plt.show()

In [ ]:
a.index

In [ ]:
BXBUS[BXBUS.route_dir==a.index]

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
ax=BXboundarygdf.plot(ax=ax,color='pink')
BXBUS[BXBUS['route_id']==a.index].plot(ax=ax,column='route_id',cmap='rainbow',legend=True)
plt.title('Top 7 Bronx Buses',size=20)
#ax.set_xlim(-73.94, -73.92)
#ax.set_ylim(40.65, 40.665)
plt.savefig('LongestBXbuses.png') 
plt.show()

In [ ]:
airshp=gp.GeoDataFrame.from_file('CHS_2009_DOHMH_2010B/CHS_2009_DOHMH_2010B.shp')

In [ ]:
airshp

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
ax=airshp.plot(ax=ax,column='asthev2',legend=True)
plt.show()


In [ ]:
AIRQUAL=pd.read_csv('data/DataPackage/Data.csv')
AIRQUAL = AIRQUAL[AIRQUAL.Measure == 'Mean']
AIRQUAL.drop(['Unique Id', 'indicator_id', 'geo_type_id', 'measurement_type_id', 'Measure',
       'internal_id', 'subtopic_id', 'description', 'geo_type_name','year_description','message', 'Unnamed: 15'], axis=1,inplace=True)
AIRQUAL.dropna(axis=0,inplace=True)
AIRQUAL.rename(columns={'geo_entity_id':'boro_cd'},inplace=True)

In [ ]:
NO = AIRQUAL[AIRQUAL['name']=='Nitric Oxide (NO)']
FPM = AIRQUAL[AIRQUAL['name']=='Fine Particulate Matter (PM2.5)']
BC = AIRQUAL[AIRQUAL['name']=='Black Carbon']
NO2_ = AIRQUAL[AIRQUAL['name']=='Nitrogen Dioxide (NO2)']
SO2_ = AIRQUAL[AIRQUAL['name']=='Sulfur Dioxide (SO2)']
O3_ = AIRQUAL[AIRQUAL['name']=='Ozone (O3)']

In [ ]:
NO.rename(columns={'data_value':'Nitric Oxide (NO)'},inplace=True)
NO.drop(['name'],axis=1,inplace=True)
FPM.rename(columns={'data_value':'PM25'},inplace=True)
FPM.drop(['name','geo_entity_name'],axis=1,inplace=True)
BC.rename(columns={'data_value':'Black Carbon'},inplace=True)
BC.drop(['name','geo_entity_name'],axis=1,inplace=True)
NO2_.rename(columns={'data_value':'Nitrgen Oxide'},inplace=True)
NO2_.drop(['name','geo_entity_name'],axis=1,inplace=True)
SO2_.rename(columns={'data_value':'SO2'},inplace=True)
SO2_.drop(['name','geo_entity_name'],axis=1,inplace=True)
O3_.rename(columns={'data_value':'Ozone'},inplace=True)
O3_.drop(['name','geo_entity_name'],axis=1,inplace=True)

In [ ]:
AIRQ = NO.merge(FPM,on='boro_cd').merge(BC,on='boro_cd').merge(NO2_,on='boro_cd').merge(SO2_,on='boro_cd').merge(O3_,on='boro_cd')
CD = CD.merge(AIRQ, on='boro_cd')
CD.plot(column='PM25',legend=True)